#Import bibliotek

In [1]:
import numpy as np
import pandas as pd
import sklearn

#Wygenerowanie danych

In [14]:
data = {
    'size': ['XL', 'L', 'M', 'L', 'M'],
    'color': ['red', 'green', 'blue', 'green', 'red'],
    'gender': ['female', 'male', 'male', 'female', 'female'],
    'price': [199.0, 89.0, 99.0, 129.0, 79.0],
    'weight': [500, 450, 300, 380, 410],
    'bought': ['yes', 'no', 'yes', 'no', 'yes']
}

df_raw = pd.DataFrame(data=data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


#Utworzenie kopii danych

In [15]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


#Zmiana typu danych i wstępna eksploracja

In [4]:
for col in ['size', 'color', 'gender', 'bought']:
  df[col] = df[col].astype('category')

#df['weight'] = df['weight'].astype('float')
df['weight'] = df.weight.astype('float')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 740.0 bytes


Wyświetlenie statystyk dla zmiennych numerycznych

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,5.0,119.0,48.476799,79.0,89.0,99.0,129.0,199.0
weight,5.0,408.0,75.299402,300.0,380.0,410.0,450.0,500.0


Wyświetlenie statystyk również dla zmiennych kategorycznych

In [6]:
df.describe(include=['category']).T

,count,unique,top,freq
size,5,3,L,2
color,5,3,green,2
gender,5,2,female,3
bought,5,2,yes,3


#LabelEncoder - mapowanie z yes na 1 i z no na 0

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['bought'])
le.transform(df['bought'])

array([1, 0, 1, 0, 1])

In [8]:
le.fit_transform(df['bought'])

array([1, 0, 1, 0, 1])

##Mapowanie - 'no' to element 0, więc 0 -> 'no'

In [9]:
le.classes_

array(['no', 'yes'], dtype=object)

In [10]:
df.bought = le.fit_transform(df.bought)
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


##Odwrócenie mapowania

In [11]:
le.inverse_transform(df.bought)

array(['yes', 'no', 'yes', 'no', 'yes'], dtype=object)

In [12]:
df.bought = le.inverse_transform(df.bought)
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


#OneHotEncoder

In [22]:
from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
encoder = OneHotEncoder(sparse=False) #wyłączenie opcji z macierzą rzadką, gdzie przechowywane są pozycje jedynek a pominięte zera (oszczędność)
encoder.fit(df[['size']])

OneHotEncoder(sparse=False)

In [23]:
encoder.transform(df[['size']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [24]:
encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

In [25]:
encoder = OneHotEncoder(drop='first', sparse=False) #Usunięcie pierwszej kolumny, aby uniknąć liniowej zależności między zmiennymi L, M, XL, np. 0, 0 -> 1; 0, 1 -> 0
encoder.fit(df[['size']])
encoder.transform(df[['size']])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [26]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


#Pandas *get_dummies()*

In [27]:
pd.get_dummies(data=df)

,price,weight,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_female,gender_male,bought_no,bought_yes
0,199.0,500,0,0,1,0,0,1,1,0,0,1
1,89.0,450,1,0,0,0,1,0,0,1,1,0
2,99.0,300,0,1,0,1,0,0,0,1,0,1
3,129.0,380,1,0,0,0,1,0,1,0,1,0
4,79.0,410,0,1,0,0,0,1,1,0,0,1


In [28]:
#drop_first, prefix
pd.get_dummies(data=df, drop_first=True, prefix='new')

,price,weight,new_M,new_XL,new_green,new_red,new_male,new_yes
0,199.0,500,0,1,0,1,0,1
1,89.0,450,0,0,1,0,1,0
2,99.0,300,1,0,0,0,1,1
3,129.0,380,0,0,1,0,0,0
4,79.0,410,1,0,0,1,0,1


In [29]:
pd.get_dummies(data=df, drop_first=True, prefix_sep='-')

,price,weight,size-M,size-XL,color-green,color-red,gender-male,bought-yes
0,199.0,500,0,1,0,1,0,1
1,89.0,450,0,0,1,0,1,0
2,99.0,300,1,0,0,0,1,1
3,129.0,380,0,0,1,0,0,0
4,79.0,410,1,0,0,1,0,1


In [30]:
#Kodowanie tylko dla wybranych kolumn
pd.get_dummies(data=df, drop_first=True, columns=['size'])

,color,gender,price,weight,bought,size_M,size_XL
0,red,female,199.0,500,yes,0,1
1,green,male,89.0,450,no,0,0
2,blue,male,99.0,300,yes,1,0
3,green,female,129.0,380,no,0,0
4,red,female,79.0,410,yes,1,0


#Standaryzacja danych - StandardScaler
Standaryzacja, aby zachować wpływ wszystkich zmiennych na model! W przypadku, gdy jedna zmienna ma większą wartość od innych, ta może mieć duży wpływ na wynik.

std() - w bibliotece pandas estymator wariancji jest nieobciążony

std() - numpy obciążony

In [34]:
# print(f"string{zmienna}")
print(f"{df['price']}\n")
print(f"Średnia: {df['price'].mean()}")
print(f"Odchylenie standardowe: {df['price'].std():.4f}")

0    199.0
1     89.0
2     99.0
3    129.0
4     79.0
Name: price, dtype: float64

Średnia: 119.0
Odchylenie standardowe: 48.4768


In [35]:
(df['price'] - df['price'].mean()) / df['price'].std()

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

In [36]:
def standardize(x):
  return (x-x.mean())/x.std()

standardize(df.price)

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

In [37]:
from sklearn.preprocessing import scale
scale(df['price'])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df[['price']])
scaler.transform(df[['price']])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [39]:
scaler = StandardScaler()
scaler.fit_transform(df[['price']])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [40]:
scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,yes
1,L,green,male,-0.691898,0.623610,no
2,M,blue,male,-0.461266,-1.603567,yes
3,L,green,female,0.230633,-0.415740,no
4,M,red,female,-0.922531,0.029696,yes


#Przygotowanie danych - wszystkie kroki

In [41]:
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [42]:
le = LabelEncoder()
df['bought'] = le.fit_transform(df['bought'])

scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])

df = pd.get_dummies(data=df, drop_first = True)
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0


In [44]:
le.classes_ #0 - no, 1 - yes

array(['no', 'yes'], dtype=object)